In [5]:
pip install numpy 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/8f/bb/aea1fbeed5b474cb8634364718abe9030d7cc7a30bf51f40bd494bbc89a2/pandas-2.0.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Obtaining dependency information for numpy>=1.21.0 from https://files.pythonhosted.org/packages/86/a1/b8ef999c32f26a97b5f714887e21f96c12ae99a38583a0a96e65283ac0a1/numpy-1.25.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 9.7 MB/s eta 0:00:000m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:01:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
replenish 0.0.12 requires ipdb, which is not installed.
replenish 0.0.12 requ

In [2]:
pip install requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/91/e6/8fa919fc84a106e9b04109de62bdf8526899e2754a64da66e1cd50ac1faa/charset_normalizer-3.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/9b/81/62fd61001fa4b9d0df6e31d47ff49cfa9de4af03adecf339c7bc30656b37/urllib3-2.0.4-py3-none-any.whl.metadata
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 5.4 MB/s eta 0:00:00
Using cached urllib3-2.0.4-py3-none-any.whl (123 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time

cuisines = ['british']

# dietary= ['vegetarian','vegan', 'gluten-free','nut-free','healthy', 'dairy-free', 'egg-free', 'low-calorie', 'low-sugar',
#           'high-protein', 'low-fat', 'high-fibre', 'keto', 'low-carb']

base_url= 'https://www.bbcgoodfood.com/search?tab=recipe'


def preference_based_search(preference):
    '''Return a url for a BBC GoodFood search that
    is specific to the input category/preference.'''
    if preference in cuisines:
        url= f'{base_url}&cuisine={preference}'
    # elif preference in dietary:
    #     url= f'{base_url}&diet={preference}'
    else:
        url=base_url
    return url

def category_specific_links(preference):
    '''Given a URL for a category search in BBC GoodFood,
    return a list of specific links for each recipe from
    the categorical search'''

    # if preference in ['vegetarian','healthy', 'gluten-free','british']:
    #     page_range=205

#     if preference == 'british':
#         page_range = 160
    
    recipe_links =[]
    for num in range(1, 80):
        try:
            search_url = preference_based_search(preference) + f'&page={num}'
            response = requests.get(search_url)
            soup = BeautifulSoup(response.content, "html.parser")
            for link in soup.find_all("a", class_="link d-block"):
                recipe_links.append(f'https://www.bbcgoodfood.com/recipes{link["href"]}')
        except:
            recipe_links.append('n')

    valid_link =[]
    for link in recipe_links:
        if 'https' not in link[10:]:
            valid_link.append(link)
    return list(set(valid_link))



def category_bbc_data(preference):
    '''Return a dataframe of recipe information for each recipe
    given a specific category'''

    individual_recipes = category_specific_links(preference)

    #list for each item retrieved
    recipe_title=[]
    prep_times =[]
    cooking_times=[]
    stars=[]
    review_count=[]
    difficulty_level=[]
    servings=[]
    description=[]
    recipe_ingredients=[]
    only_ingredients=[]

    #here the individual recipe links are pulled and iterated
    for recipe in individual_recipes:
        time.sleep(0.25)
        response = requests.get(recipe, allow_redirects=False)
        soup = BeautifulSoup(response.content, "html.parser")

        #recipe title
        try:
            title = soup.find("div", class_= "headline post-header__title post-header__title--masthead-layout").find(
            "h1",class_='heading-1').string
            recipe_title.append(title)
        except:
            recipe_title.append('n')

        #prep and cooking times
        try:
            times = soup.find("div", class_= "icon-with-text__children").find_all('time')
            prep_times.append(times[0].text.strip(' mins'))
        except:
            prep_times.append('n')

        try:
            times = soup.find("div", class_= "icon-with-text__children").find_all('time')
            cooking_times.append(times[1].text.strip(' mins'))
        except:
            cooking_times.append('n')


        try:
        #star ratings
            star_rating = soup.find("div", class_= "rating__values").find("span",class_='sr-only').string
            star_rating_float = float(star_rating.strip('A star rating of ').strip(' out of 5.'))
            stars.append(star_rating_float)
        except:
            stars.append('n')

        try:
        #review count
            num_reviews = soup.find("div", class_= "rating__values").find("span",class_='rating__count-text body-copy-small').string
            num_reviews_float = float(num_reviews.strip(' ratings'))
            review_count.append(num_reviews_float)
        except:
            review_count.append('n')

        #servings
        try:
            serving = soup.find("div", class_= "icon-with-text post-header__servings body-copy-small body-copy-bold icon-with-text--aligned").find("div", class_= "icon-with-text__children").string
        #servings_int = int(serving.strip('Serves '))
            servings.append(serving)
        except:
            servings.append("None")

        #description
        try:
            tagline = soup.find("div", class_= "editor-content mt-sm pr-xxs hidden-print").find("p").string
            description.append(tagline)
        except:
            description.append('n')


        #difficulty level
        try:
            difficulty = soup.find("div", class_= "icon-with-text post-header__skill-level body-copy-small body-copy-bold icon-with-text--aligned").find("div", class_= "icon-with-text__children").string
            difficulty_level.append(difficulty)
        except:
            difficulty_level.append('n')


        #ingredients with measurement
        try:
            for ingredient_group in soup.find_all('section', class_='recipe__ingredients col-12 mt-md col-lg-6'):
                ingredients = ingredient_group.find_all('li')
                ingredient_text = ''
            for ingredient in ingredients:
                ingredient_text += ingredient.get_text(',') + ', '
            recipe_ingredients.append(ingredient_text)
        except:
            recipe_ingredients.append('n')

        #ingedients alone
        try:
            for ingred_group in soup.find_all('section', class_='recipe__ingredients col-12 mt-md col-lg-6'):
                ingredient = ingred_group.find_all('a')
                ingredient_text = ''
            for ing in ingredient:
                ingredient_text += ing.get_text().strip(',') + ', '
            only_ingredients.append(ingredient_text)
        except:
            only_ingredients.append('n')

        #Creating a Dictionary
    category_dictionary= {
        'recipe_title':recipe_title,
        'stars':stars,
        'prep_times':prep_times,
        'cooking_times': cooking_times,
        'review_count':review_count,
        'difficulty_level':difficulty_level,
        'servings':servings,
        'description':description,
        'specific_ingredients':recipe_ingredients,
        'ingredients':only_ingredients
    }

    #Dictionary to dataframe
    df = pd.DataFrame.from_dict(category_dictionary)

    return df

def get_image(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    image_url = soup.find('img')
    pass



def load_data():
    '''merging the dataframes together for the entire dataframe used in
    the replenish modelling and project'''

    # column_names = ['recipe_title','stars','prep_times','cooking_times', 'review_count',
    #                  'difficulty_level','servings','description', 'specific_ingredients' 'ingredients', 'preference']
    # final_df = pd.DataFrame(columns=column_names)

    print("Scraping cuisines from BBC Good Foods")
    all_df_lst = []

    for cuisine in cuisines:
        df = category_bbc_data(cuisine)
        df['preference']=cuisine
        all_df_lst.append(df)

    # final_df = pd.concat(all_df_lst)
    print('Scraping done')
    # print("Scraping dietary recipes from BBC Good Foods")

    # for diet in dietary:
    #     df = category_bbc_data(diet)
    #     df['preference']=diet
    #     all_df_lst.append(df)

    final_df = pd.concat(all_df_lst)

    final_df.reset_index(inplace=True)
    print('Finished scraping all')

    return final_df

In [7]:
df_british_first_80 = load_data()
df_british_first_80.to_csv('british_80.csv')

Scraping cuisines from BBC Good Foods
Scraping done
Finished scraping all


In [8]:
df_british_first_80

,index,recipe_title,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference
0,0,Chicken soup,4.4,15,35,180.0,Easy,Serves 4,Use up leftover chicken in this rustic soup wi...,"1 tbsp ,olive oil, 2 ,onions,, chopped, 3 medi...","olive oil, onions, carrots, thyme leaves, left...",british
1,1,Creamy pumpkin & lentil soup,4.0,15,35,74.0,Easy,Serves 4,Whether you're carving a Halloween pumpkin or ...,"1 tbsp ,olive oil,, plus 1 tsp, 2 ,onions,, ch...","olive oil, onions, garlic cloves, chopped pump...",british
2,2,Citrus bars,4.0,20,35,18.0,More effort,Makes 18,Crisp shortbread biscuit base with a creamy ci...,"250g ,plain flour, 85g ,icing sugar,, plus ext...","plain flour, icing sugar, butter, lemons, larg...",british
3,3,Spiced plum & blackberry crumble,4.3,20,1 hr and 30,14.0,Easy,Serves 6,"Showcase fruit in a piping hot, oat-topped cru...","140g ,plain flour, 140g ,butter,, cut into sma...","plain flour, butter, soft brown sugar, ripe pl...",british
4,4,Gooseberry & elderflower jam,n,5,30,7.0,More effort,Makes 3 x 340ml jars,Preserve the taste of the season and use up a ...,"1kg ,gooseberries, 900g ,granulated white suga...","gooseberries, granulated white sugar,",british
...,...,...,...,...,...,...,...,...,...,...,...,...
2012,2012,Lemon drizzle scones,4.3,20,12,16.0,Easy,Makes 6,We've combined two of your favourite baking re...,"250g ,self-raising flour,, plus a little extra...","self-raising flour, butter, golden caster suga...",british
2013,2013,Maple & pecan carrots,n,10,40,7.0,Easy,Serves 8,Sweet maple syrup and pecan nuts go brilliantl...,"1kg ,carrot,, quartered, 1 tbsp ,olive oil, 4 ...","carrot, olive oil, maple syrup, pecans,",british
2014,2014,Prawn & mussel pies,4.4,10,15,5.0,Easy,Serves 4,A soupy pie with light pastry and seafood fill...,"half x , pack ready-rolled puff pastry, 150ml ...","mussels, olive oil, large onion, sticks cele...",british
2015,2015,Sticky sausage baguettes,4.7,5,30,6.0,Easy,Makes 8,Jazz up your bangers with a sweet honey mustar...,"8 ,pork sausages, 1 tbsp ,oil, 3 tbsp ,soy sau...","soy sauce, honey, wholegrain mustard,",british
